## Intégration des vacances et des fêtes

In [1]:
import os
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from describe import stats_histo
import problem
import warnings
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

On importe les données initiales

In [2]:
X_train, y_train = problem.get_train_data()

In [3]:
X_train.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd
0,2012-06-19,ORD,DFW,12.875000,9.812647
1,2012-09-10,LAS,DEN,14.285714,9.466734
2,2012-10-05,DEN,LAX,10.863636,9.035883
3,2011-10-09,ATL,ORD,11.480000,7.990202
4,2012-02-21,DEN,SFO,11.450000,9.517159


In [6]:
holiday = pd.read_csv("data/us-federal-holidays-2011-2020.csv")

In [7]:
holiday.head()

,Date,Holiday
0,12/31/2010,New Year's Day
1,1/17/2011,"Birthday of Martin Luther King, Jr."
2,2/21/2011,Washington's Birthday
3,5/30/2011,Memorial Day
4,7/4/2011,Independence Day


In [8]:
X_train["DateOfDeparture"] = pd.to_datetime(X_train.DateOfDeparture)

# - - - - - - - 

holiday = holiday.rename(columns={'Date': 'DateOfDeparture'})
holiday["DateOfDeparture"] = pd.to_datetime(holiday["DateOfDeparture"])

X_train = pd.merge(X_train, holiday, 
              how='left',
              on=["DateOfDeparture"],
             sort=False)

X_train["Holiday"] = pd.get_dummies(X_train.Holiday.isnull().astype(int))

In [9]:
X_train["Y"] = y_train

L'ANOVA ou test de Student dans le cas de deux échantillons teste l'hypothèse nulle que un ou plusieurs groupes ont la même moyenne.

In [33]:
from scipy.stats import f_oneway, ttest_ind

test = ttest_ind(X_train["Y"][X_train["Holiday"] == 1],
                 X_train["Y"][X_train["Holiday"] == 0])

print("Statistique de test :",test.statistic)
print("P-valeur :",test.pvalue.round(4))

Statistique de test : -6.329835202510881
P-valeur : 0.0


In [10]:
X_train.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,Holiday,Y
0,2012-06-19,ORD,DFW,12.875000,9.812647,0,12.331296
1,2012-09-10,LAS,DEN,14.285714,9.466734,0,10.775182
2,2012-10-05,DEN,LAX,10.863636,9.035883,0,11.083177
3,2011-10-09,ATL,ORD,11.480000,7.990202,0,11.169268
4,2012-02-21,DEN,SFO,11.450000,9.517159,0,11.269364


On va ajouter les données sur `external_data`

In [13]:
extern = pd.read_csv('submissions/submit/external_data.csv', index_col=0)

On regarde le nom des colonnes pour le merge : 

In [14]:
extern.head(2)

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,CloudCover,Events,WindDirDegrees,Holiday,TOTAL_DEPARTURE,TOTAL_DESTINATION,unemployment,INDEX_PRICE,Index_passengers,Price
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,3,Sun,129,0,3582088,3580851,10.995779,0.151846,115.0,85.52
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,2,Sun,185,0,3582088,3580851,10.995779,0.151846,115.0,85.52


In [17]:
holiday.head(2)

,Date,Holiday
0,2010-12-31,New Year's Day
1,2011-01-17,"Birthday of Martin Luther King, Jr."


In [18]:
extern["Date"] = pd.to_datetime(extern.Date)

# - - - - - - - 

holiday = holiday.rename(columns={'Date': 'Date'})
holiday["Date"] = pd.to_datetime(holiday["Date"])

extern = pd.merge(extern, holiday, 
              how='left',
              on=["Date"],
             sort=False)

extern["Holiday"] = pd.get_dummies(extern.Holiday.isnull().astype(int))

On vérifie : 

In [58]:
extern[extern.Holiday == 1].iloc[10:11,:]

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Holiday
248,2011-09-05,LAX,27,22,17,15,14,13,84,68,52,1012,1010,1009,16,16,13,29,12,37.0,T,4,Rain,243,1


In [59]:
holiday[holiday.Date == '2011-09-05']

,Date,Holiday
5,2011-09-05,Labor Day


On peut utiliser ce `csv` pour notre jeu de données.

In [60]:
extern.to_csv('submissions/submit/external_data.csv')

In [141]:
import datetime

In [148]:
def calcul_date(x):
    day_bef = datetime.timedelta(days=-2) + x
    day_aft = datetime.timedelta(days=+2) + x
    return pd.Series([day_aft,day_bef])

day_wind = holiday.Date.apply(calcul_date)

In [149]:
close_holiday = pd.concat([day_wind.iloc[:,0],day_wind.iloc[:,1]])

In [150]:
close_holi = pd.DataFrame(close_holiday, columns=["Date"])
close_holi.Date = pd.to_datetime(close_holi.Date)
close_holi["Holiday"] = "Close to holiday two"

In [151]:
close_holi

,Date,Holiday
0,2011-01-02,Close to holiday two
1,2011-01-19,Close to holiday two
2,2011-02-23,Close to holiday two
3,2011-06-01,Close to holiday two
4,2011-07-06,Close to holiday two
...,...,...
95,2020-09-05,Close to holiday two
96,2020-10-10,Close to holiday two
97,2020-11-09,Close to holiday two
98,2020-11-24,Close to holiday two


In [153]:
Holiday_closeness = pd.concat([close_holi, Holiday_data]).sort_values(by='Date')

In [157]:
Holiday_closeness.to_csv('data/Holiday_closeness.csv', index=False)

## Ajout dans external data

In [3]:
holiday_close = pd.read_csv('data/Holiday_closeness.csv')

In [7]:
def sem(x):
    if x == "Close to holiday two":
        return 1
    elif x == "Close to holiday one":
        return 2
    else : 
        return 3


holiday_close.Holiday = holiday_close.Holiday.apply(sem)

In [15]:
extern = pd.read_csv('submissions/submit/external_data.csv', index_col=0)

In [16]:
extern["Date"] = pd.to_datetime(extern.Date)

# - - - - - - - 
holiday_close["Date"] = pd.to_datetime(holiday_close["Date"])

extern = pd.merge(extern, holiday_close, 
              how='left',
              on=["Date"],
             sort=False).drop(columns="Holiday_x")

In [19]:
extern["Holiday_y"] = extern["Holiday_y"].fillna(0)

In [23]:
extern = extern.rename(columns={"Holiday_y":"Holiday"})

In [25]:
extern.to_csv('submissions/submit/external_data.csv')